# Probabilistic Models – Spring 2022
## Exercise Session 2
Return by Feb 8th 12.00 through Moodle. Session on Feb 8th 14.15.

<span style="color:red">**TYPE YOUR NAME HERE**</span>

### Instructions
Make sure the notebook produces correct results when ran sequentially starting from the first cell. You can ensure this by clearing all outputs (`Edit > Clear All Outputs`), running all cells (`Run > Run All Cells`), and finally correcting any errors.

To get points:
1. Submit your answers to the automatically checked Moodle test. 
 - You have 5 tries on the test: the highest obtained score will be taken into account.
 - For numerical questions the tolerance varies by question and will be specified in Moodle.
2. Submit this notebook containing your derivations to Moodle.

Some of the exercises will ask you to return a DAG as an answer. To make it possible to evaluate the answer automatically in Moodle use the following format. Construct the DAG as an adjacency matrix where $A[i, j] = 1$ if there is an edge $j \rightarrow i$ and 0 otherwise. The nodes should be in alphabetical order, so $A \rightarrow B$ corresponds to $0 \rightarrow 1$ (or $1 \rightarrow 2$ in R's 1-based indexing). Finally, concatenate all the rows starting from the first one and submit the vector as your answer. For example the DAG $A \rightarrow B \leftarrow C$ is encoded by the matrix 

$$\begin{bmatrix} 0 & 0 & 0 \\ 1 & 0 & 1 \\ 0 & 0 & 0 \end{bmatrix}$$

and vector $000101000$.

You can make use of the following examples to construct the DAG with Python/R.

In [2]:
# RUN THIS IF WORKING IN PYTHON
import pandas as pd

# Function to concatenate matrix rows into a single string
def mat2vec(dag):
    return ''.join(str(x) for x in dag.values.reshape(dag.values.shape[0]**2))

# Adjacency matrix
rvs = ["A", "B", "C"]
DAG = pd.DataFrame(0, index=rvs, columns=rvs)

# Example: Set parents of B to be A and C.
DAG.loc["B", ["A", "C"]] = 1

# Create the vector
print(mat2vec(DAG))

000101000


In [2]:
# RUN THIS IF WORKING IN R

# Function to concatenate matrix rows into a single string
mat2vec <- function(dag) {
    return(paste(apply(dag, 1, paste, collapse=""), collapse=""))
}

# Adjacency matrix
rvs <- c("A", "B", "C")
DAG <- data.frame(matrix(0L, ncol = 3, nrow = 3))
colnames(DAG) <- rvs
rownames(DAG) <- rvs

# Example: Set parents of B to be A and C.
DAG["B", c("A", "C")] <- 1

# Create the vector
cat(mat2vec(DAG))

000101000

## Exercise 1
***

Let us consider a 4-sided dice rolling experiment as a multinomial model (i.i.d.   multi-valued Bernoulli). We roll the dice 20 times, and observe data $D$ with the following counts for the sides:

In [21]:
!cat data/1.csv

side	counts
1	5
2	3
3	7
4	5


(a) Calculate the maximum likelihood parameters, given the above data.

(b) Calculate the posterior distribution $P(\theta_1, \theta_2, \theta_3, \theta_4 | D)$ considering the prior $Dir(\alpha_1, \alpha_2, \alpha_3, \alpha_4)$, with
- $\alpha_1 = \alpha_2 = \alpha_3 = \alpha_4 = 1$, i.e., the uniform prior and
- $\alpha_1 = \alpha_2 = \alpha_3 = \alpha_4 = 0.5$, i.e., the Jeffrey's prior.

For both, report the mean. 

(c) Using Bayesian inference with the uniform prior (above), calculate the predictive distribution (all 4 probabilities) of the next result given $D$.

(d) Let $\alpha_4$, the 4th hyperparameter to the Dirichlet prior be 3. Specify $\alpha_1$, $\alpha_2$ and $\alpha_3$ such that the mode of the posterior distribution is at $\theta_1 = \theta_2 = \theta_3 = \theta_4$.


In [ ]:
# Provide your answer in cells here

## Exercise 2
***

Show by using the d-separation criterion that a node in a Bayesian network is conditionally independent of all the other nodes, given its (minimal) Markov blanket (parents, children, spouses (parents of children)). 

Give the answer verbally in Moodle. It will be checked manually. For your own future reference it's a good idea to paste the answer here, too. 

Some correct and incorrect proofs will be posted anonymously in Moodle for study, by returning you agree that your version may be published anonymously. 

In [ ]:
# Provide your answer in cells here

## Exercise 3
***

Consider the following BN structure. Answer the following queries and questions, justifying your answers.

![](2.3.svg)

(a) Decide whether the following d-separations hold or not. 
- $G \mathrel{⫫}_{G} D \mid A, E$
- $D \mathrel{⫫}_{G} F$
- $H \mathrel{⫫}_{G} B \mid G, C$
- $G \mathrel{⫫}_{G} H \mid A, F$

(b) Construct a Markov equivalent DAG (other than the given), and return it to Moodle in the format specified at the top of the notebook. How many equivalent DAGs are there in total (including the given one)?

(c) Suppose all variables in this network are binary. How many free parameters are needed to parameterize this network?

In [ ]:
# Provide your answer in cells here

## Exercise 4
***

Consider again the DAG in Exercise 3.

a) What is the factorization implied by the DAG?

Return the factorization in Moodle in plain text in the exact same format as the example: `P(A,B,C)P(B,C|D)P(C|E,F)`. Here
- a set of variables $\{ B, A, C \}$ is encoded as `A,B,C` (note the alphabetical order);
- the factors themselves are in alphabetical order, so not `P(B)P(A)` but `P(A)P(B)`, not `P(A|C)P(A|B)` but `P(A|B)P(A|C)`.

b) Which of the following independencies are stated by the local Markov condition asserted by the DAG?

- $D \mathrel{⫫} F$
- $E \mathrel{⫫} A \mid H$
- $B \mathrel{⫫} H \mid F, G$

In [12]:
# Provide your answer in cells here

## Exercise 5
***

Consider the following DAG: $X \rightarrow Y \rightarrow Z$.

(a) Suppose the variables are binary and another, equivalent DAG encodes the same joint distribution with the following parameters:

\begin{aligned}
P(Y = 1) = 0.3 \\
P(X = 1 | Y = 1) = 0.2 \\
P(X = 1 | Y = 0) = 0.8 \\
P(Z = 1 | Y = 1) = 0.8 \\
P(Z = 1 | Y = 0) = 0.2 \\
\end{aligned}

Give the parameters corresponding to the first DAG.

(b) What values do the variables take at the mode of the joint distribution?

(c) Compute the marginal probabilities $P(X)$, $P(Y)$, $P(Z)$ and their respective most probable arguments (which value for each random variable gets the highest probability).

In [ ]:
# Provide your answer in cells here

## Exercise 6
***

Faithfulness. Consider a DAG $X \rightarrow Y$ over binary random variables $X,Y$.

(a) Give parameters for a BN over the DAG such that we have $X \mathrel{⫫} Y$ (conditional independence) and the distribution is positive, i.e., gives a non-zero probability for all assignments of $X,Y$.

(b) Give parameters for a BN over the DAG such that we have $X \not\mathrel{⫫} Y$ such that the distribution is positive as well.

(c) Take the parameters in (a), add small random noise to the parameters and renormalize the probabilities such that you have a (valid) BN (i.e. rows of CPTs should sum up to one and all probabilities should be positive). Do you still have $X \mathrel{⫫} Y$?

(d) Does any of this contradict the soundness and completeness of d-separation? Why?

For each part give a short verbal answer in Moodle, e.g., "P(X = 1) = x, P(Y = 1 | X = 1) = y, P(Y = 1 | X = 0) = z  ...". They will be graded manually.

In [ ]:
# Provide your answer in cells here